In [79]:
import numpy as np
from scipy.stats import gamma
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

class KCI:
 
    def kernel(self, x, y):
        T = len(y)

        x = np.array(x)
        y = np.array(y)
        x = x - np.mean(x)
        x = x / np.std(x)
        y = y - np.mean(y)
        y = y / np.std(y)

        if T < 200:
            width = 0.8
        elif T < 1200:
            width = 0.5
        else:
            width = 0.3

        theta = 1 / (width**2)

        Kx = 1.0 * RBF(theta).__call__(x, x)
        Ky = 1.0 * RBF(theta).__call__(y, y)

        return Kx, Ky


    def statistic(self, x, y):

        T = len(y)
        
        H = np.eye(T) - np.ones((T, T)) / T

        Kx, Ky = self.kernel(x, y)

        Kx = np.matmul(np.matmul(H, Kx), H)
        Ky = np.matmul(np.matmul(H, Ky), H)

        stat = np.trace(np.matmul(Kx, Ky))

        return stat

    def test(self, x, y):

        T = len(y)
        Kx, Ky = self.kernel(x, y)
        stat = self.statistic(x, y)

        mean_appr = (np.trace(Kx) * np.trace(Ky)) / T
        var_appr = 2 * np.trace(np.matmul(Kx, Kx)) * np.trace(np.matmul(Ky, Ky)) / T**2
        k_appr = mean_appr**2 / var_appr
        theta_appr = var_appr / mean_appr
        pvalue = 1 - np.mean(gamma.cdf(stat, k_appr, theta_appr))

        self.stat = stat

        return stat, pvalue

In [129]:
from hyppo.tools import linear, power, joint_normal, multimodal_independence, logarithmic, exponential

In [138]:
np.random.seed(123456789)
n = 100
x, y = linear(n, 1)
kci = KCI()
stat, pvalue = kci.test(x, y)

print('Stat:', stat, 'pvalue:', pvalue)

Stat: 544.6911482512229 pvalue: 0.0


In [139]:
np.random.seed(123456789)
n = 100
x, y = joint_normal(n, 1)
kci = KCI()
stat, pvalue = kci.test(x, y)

print('Stat:', stat, 'pvalue:', pvalue)

Stat: 105.75819757867163 pvalue: 7.638334409421077e-14


In [140]:
np.random.seed(123456789)
n = 100
x, y = multimodal_independence(n, 1)
kci = KCI()
stat, pvalue = kci.test(x, y)

print('Stat:', stat, 'pvalue:', pvalue)

Stat: 9.298485426452766 pvalue: 1.0


In [141]:
np.random.seed(123456789)
n = 100
x, y = logarithmic(n, 1)
kci = KCI()
stat, pvalue = kci.test(x, y)

print('Stat:', stat, 'pvalue:', pvalue)

Stat: 88.52619309350321 pvalue: 0.0002686652063370598


In [142]:
np.random.seed(123456789)
n = 100
x, y = exponential(n, 1)
kci = KCI()
stat, pvalue = kci.test(x, y)

print('Stat:', stat, 'pvalue:', pvalue)

Stat: 452.8347492253364 pvalue: 0.0
